In [ ]:
import sys
import matplotlib . pyplot as plt
import numpy as np
from netCDF4 import Dataset as NetCDFFile

filename = "Data1/W_XX-EUMETSAT-Darmstadt,VIS+IR+HRV+IMAGERY,MSG4+SEVIRI_C_EUMG_20200921000010.nc"

In [ ]:
def kdtree ( self , latvar , lonvar ) :
    rad_factor = pi / 180.0 # for trignometry , need angles in radians
    latvals = latvar [:] * rad_factor
    lonvals = lonvar [:] * rad_factor
    shape = latvals . shape
    clat , clon = cos ( latvals ) , cos ( lonvals )
    slat , slon = sin ( latvals ) , sin ( lonvals )
    clat_clon = clat * clon
    clat_slon = clat * slon
    triples = list ( zip ( np . ravel ( clat * clon ) , np . ravel ( clat * slon ) , np . ravel ( slat ) ))
    return cKDTree ( triples )


In [ ]:
# Constants
no_data_value = 0

# Reflectance factor for visible bands
HRV_F = 25.15
VIS006_F = 20.76
VIS008_F = 23.30
IR_016_F = 19.73

# Calibration coefficients from
# ’A Planned Change to the MSG Level 1.5 Image Product Radiance Definition ’
# ,
# " Conversion from radiances to reflectances for SEVIRI warm channels "
# EUM/ MET/ TEN /12/0332
# , and
# "The Conversion from Effective Radiances to Equivalent Brightness
# Temperatures "
# EUM/ MET/ TEN /11/0569

CALIB = {}

# Meteosat 10

CALIB [323] = {'HRV': {'F': 78.9416 / np . pi },
               'VIS006': {'F': 65.5148 / np . pi } ,
               'VIS008': {'F': 73.1807 / np . pi } ,
               'IR_016': {'F': 62.0208 / np . pi } ,
               'IR_039': {'VC': 2547.771 ,
                           'ALPHA': 0.9915 ,
                           'BETA': 2.9002} ,
               'WV_062': {'VC': 1595.621 ,
                           'ALPHA': 0.9960 ,
                           'BETA': 2.0337} ,
               'WV_073': {'VC': 1360.337 ,
                           'ALPHA': 0.9991 ,
                           'BETA': 0.4340} ,
               'IR_087': {'VC': 1148.130 ,
                           'ALPHA': 0.9996 ,
                           'BETA': 0.1714} ,
               'IR_097': {'VC': 1034.715 ,
                           'ALPHA': 0.9999 ,
                           'BETA': 0.0527} ,
               'IR_108': {'VC': 929.842 ,
                           'ALPHA': 0.9983 ,
                           'BETA': 0.6084} ,
               'IR_120': {'VC': 838.659 ,
                           'ALPHA': 0.9988 ,
                           'BETA': 0.3882} ,
               'IR_134': {'VC': 750.653 ,
                           'ALPHA': 0.9982 ,
                           'BETA': 0.5390}}

# Polynomial coefficients for spectral - effective BT fits
BTFIT_A_IR_039 = 0.0
BTFIT_A_WV_062 = 0.00001805700
BTFIT_A_WV_073 = 0.00000231818
BTFIT_A_IR_087 = -0.00002332000
BTFIT_A_IR_097 = -0.00002055330
BTFIT_A_IR_108 = -0.00007392770
BTFIT_A_IR_120 = -0.00007009840
BTFIT_A_IR_134 = -0.00007293450

BTFIT_B_IR_039 = 1.011751900
BTFIT_B_WV_062 = 1.000255533
BTFIT_B_WV_073 = 1.000668281
BTFIT_B_IR_087 = 1.011803400
BTFIT_B_IR_097 = 1.009370670
BTFIT_B_IR_108 = 1.032889800
BTFIT_B_IR_120 = 1.031314600
BTFIT_B_IR_134 = 1.030424800

BTFIT_C_IR_039 = -3.550400
BTFIT_C_WV_062 = -1.790930
BTFIT_C_WV_073 = -0.456166
BTFIT_C_IR_087 = -1.507390
BTFIT_C_IR_097 = -1.030600
BTFIT_C_IR_108 = -3.296740
BTFIT_C_IR_120 = -3.181090
BTFIT_C_IR_134 = -2.645950


C1 = 1.19104273 *10**(-16)
C2 = 0.0143877523

# Class _Calibrator

eval_np = eval
log = np . log
class Calibrator ( object ) :
    """
    Calibrator for Level 1.5 SEVIRI Images , the expected format for ‘hdr ‘ is
    either a NetCDF4 Dataset or a HDF5 group , before METADATA or DATA
    bifurcation . On the other hand , ‘channel_name ‘ is expected to be a string
    containing the name of the cannel , for instance ’VIS_006 ’.
    """
    def __init__ ( self , hdr , channel_name ):
        self . hdr = hdr
        self . instance = type ( hdr )
        self . channel_name = channel_name
        
    def __call__ ( self , calibrate ='reflectances /bt', channel_name = None ):
        """
        Computes the radiances and reflectances /bt of a given channel . The
        110 * calibrate * argument should be set to ’counts ’ for no calibration ,
        ’ reflectances /bt ’ for default reflectances /bt calibration , and ’radiances ’
        112 for returning radiances . The default value is 1.
        """
        hdr = self . hdr
        channel_name = self . channel_name if not channel_name else channel_name
        calibrate = {'counts': 0, 'reflectances /bt': 1, 'radiances': 2}[ calibrate ]
        
        if self . instance == Dataset :
            channels = {" VIS006 ": 'ch1 ',
                        " VIS008 ": 'ch2 ',
                        " IR_016 ": 'ch3 ',
                        " IR_039 ": 'ch4 ',
                        " WV_062 ": 'ch5 ',
                        " WV_073 ": 'ch6 ',
                        " IR_087 ": 'ch7 ',
                        " IR_097 ": 'ch8 ',
                        " IR_108 ": 'ch9 ',
                        " IR_120 ": 'ch10',
                        " IR_134 ": 'ch11'}
        else :
            channels = {" VIS006 ": 'Channel 01',
                        " VIS008 ": 'Channel 02',
                        " IR_016 ": 'Channel 03',
                        " IR_039 ": 'Channel 04',
                        " WV_062 ": 'Channel 05',
                        " WV_073 ": 'Channel 06',
                        " IR_087 ": 'Channel 07',
                        " IR_097 ": 'Channel 08',
                        " IR_108 ": 'Channel 09',
                        " IR_120 ": 'Channel 10',
                        " IR_134 ": 'Channel 11',
                        " HRV ": 'Channel 12'}
            
        chn_nb = channels [ channel_name ]
        chn_index = list ( channels . values () ). index ( chn_nb )
        planned_chan_processing = hdr ['METADATA'][ 'HEADER'][ 'ImageDescription'][ 'ImageDescription_DESCR'][26][1]. split (',') 
        
        if self.instance is not Dataset :
            
        else hdr . variables ['planned_chan_processing'][:]
                
        cal_type = [int (d) for d in planned_chan_processing ]
        
        image = hdr ['DATA'][ chn_nb ][ 'IMAGE_DATA'][:] if self . instance is not Dataset
            else hdr . variables [ chn_nb ][:]
        
        if calibrate == 0:
            return ( image ,
                        " counts ")
        
        mask = ( image == no_data_value )
        
        cal = hdr ['METADATA'][ 'HEADER'][ 'RadiometricProcessing']['Level15ImageCalibration_ARRAY']
                
        if self . instance is not Dataset
        else [[1 , 0] for chn in channels . values () ]
            
        cslope = cal [ chn_index ][0]
        coffset = cal [ chn_index ][1]
        
        radiances = eval_np ('image * cslope + coffset')
        radiances [ radiances < 0] = 0
        
        if calibrate == 2:
            return ( np . ma . MaskedArray ( radiances , mask = mask ) ,
                    "mW m -2 sr -1 (cm -1) -1")
        
        sat = 323 # Meteosat 10 hdr [" SatelliteDefinition "][" SatelliteId "]
        if sat not in CALIB :
            raise Exception ("No calibration coefficients available for "
                                    + " this satellite (" + str ( sat ) + ")")
        
        # Reflectance percentage
        if channel_name in ["HRV ", " VIS006 ", " VIS008 ", " IR_016 "]:
            solar_irradiance = CALIB [ sat ][ channel_name ]["F"]
            reflectance = eval_np ('( radiances / solar_irradiance ) * 100.')
            return ( np . ma . MaskedArray ( reflectance , mask = mask ) ,
                    "%")
        
        # Brightness temperature
        chn_nb = chn_index
        wavenumber = CALIB [ sat ][ channel_name ]["VC"]
        
        if cal_type [ chn_nb ] == 2:
            # computation based on effective radiance
            alpha = CALIB [ sat ][ channel_name ][" ALPHA "]
            beta = CALIB [ sat ][ channel_name ][" BETA "]
            
            cal_data = eval_np ('(( C2 * 100. * wavenumber / '
                                'log(C1 * 1.0 e6 * wavenumber ** 3 / '
                                '(1.0e -5 * radiances ) + 1)) - beta ) / alpha ')
        
        elif cal_type [ chn_nb ] == 1:
            # computation based on spectral radiance
            cal_data = eval_np ('C2 * 100. * wavenumber / '
                                'log (C1 * 1.0 e6 * wavenumber ** 3 / '
                                '(1.0e -5 * radiances ) + 1)')
            
            coef_a = eval (" BTFIT_A_ " + channel_name )
            coef_b = eval (" BTFIT_B_ " + channel_name )
            coef_c = eval (" BTFIT_C_ " + channel_name )
            
            cal_data = eval_np (( 'cal_data ** 2 * coef_a + '
                                  'cal_data * coef_b + coef_c '))

        mask = mask | np . isnan ( cal_data ) | np . isinf ( cal_data )
        cal_data = np . ma . MaskedArray ( np . array ( cal_data ) , mask = mask )
        return ( cal_data ,
                "K")


In [ ]:
def convertNetCDF ( path , file ):
    ncfile = Dataset (file , ’r’)
    ns = KDTree . KDTree ( ncfile , ’lat ’, ’lon ’)
    cal = _Calibrator . Calibrator ( ncfile , " VIS006 ")
    calibration_modes = [’radiances ’, ’ reflectances /bt ’]
    channels = {" VIS006 ": 'ch1',
                " VIS008 ": 'ch2 ',
                " IR_016 ": 'ch3 ',
                " IR_039 ": 'ch4 ',
                " WV_062 ": 'ch5 ',
                " WV_073 ": 'ch6 ',
                " IR_087 ": 'ch7 ',
                " IR_097 ": 'ch8 ',
                " IR_108 ": 'ch9 ',
                " IR_120 ": 'ch10 ',
                " IR_134 ": 'ch11 '}
    data = {}
    for chn_nb in channels . keys () :
        data [ chn_nb ] = {}
            for calibration in calibration_modes :
                data [ chn_nb ][ calibration ] = cal ( calibrate = calibration , channel_name = chn_nb )

    grid = list ( ns . querySubset (44.25 , -9.5 , 35.5 , 4.75 , 0.125) )
    
    myp = []
    for ( lat , lon ) , index in grid :
        row = [ lon , lat ]
            for chn_name in sorted ( channels . keys () ):
                calibration in calibration_modes :
                    row . append ( data [ chn_name ][ calibration ][0][ index ])
                    myp . append ( row )
    myp = np . matrix ( myp )

    filedate = ncfile . wmo_filename . split (’_’) [ -1]. split (’.’) [0][: -4]
    np . save ( path + filedate + ’. seviri . myp ’, myp ) # hourly compressed np files